## RNN Modeling of Behavior and Performance

- In this notebook, we will be experimenting with two models: skill2skill and DKT (Deep Knowledge Tracing). 
- Starter code for both the models has already been provided to you. 
- See below for how to initiate and train the model.

In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn

You should consider upgrading via the '/Users/elipleaner/workspace/mled/final/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/elipleaner/workspace/mled/final/env/bin/python3 -m pip install --upgrade pip' command.
  Using cached sklearn-0.0-py2.py3-none-any.whl
You should consider upgrading via the '/Users/elipleaner/workspace/mled/final/env/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from sklearn.model_selection import train_test_split

In [1]:
response_df = pd.read_csv('correct.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
assistment_df = pd.read_csv('assistment_id.tsv', sep='\t').drop('Unnamed: 0', axis=1)
skill_dict = {}
with open('skill_dict.json', 'r', encoding='utf-8') as f:
    loaded = json.load(f)
    for k, v in loaded.items():
        skill_dict[k] = int(v)

skill_num = len(skill_dict) + 1 # including 0

def one_hot(skill_matrix, vocab_size):
    '''
    params:
        skill_matrix: 2-D matrix (student, skills)
        vocal_size: size of the vocabulary
    returns:
        a ndarray with a shape like (student, sequence_len, vocab_size)
    '''
    seq_len = skill_matrix.shape[1]
    result = np.zeros((skill_matrix.shape[0], seq_len, vocab_size))
    for i in range(skill_matrix.shape[0]):
        result[i, np.arange(seq_len), skill_matrix[i]] = 1.
    return result

def dkt_one_hot(skill_matrix, response_matrix, vocab_size):
    seq_len = skill_matrix.shape[1]
    skill_response_array = np.zeros((skill_matrix.shape[0], seq_len, 2 * vocab_size))
    for i in range(skill_matrix.shape[0]):
        skill_response_array[i, np.arange(seq_len), 2 * skill_matrix[i] + response_matrix[i]] = 1.
    return skill_response_array

def preprocess(skill_df, response_df, skill_num):
    skill_matrix = skill_df.iloc[:, 1:].values
    response_array = response_df.iloc[:, 1:].values
    skill_array = one_hot(skill_matrix, skill_num)
    skill_response_array = dkt_one_hot(skill_matrix, response_array, skill_num)
    return skill_array, response_array, skill_response_array
    

skill_array, response_array, skill_response_array = preprocess(skill_df, response_df, skill_num)

NameError: name 'pd' is not defined

In [ ]:
# Q1
print(df.skill_name.value_counts().head())
print(df.skill_name.value_counts().tail())

skill_id = skill_dict[df.skill_name.value_counts().index[0]]

response_count = skill_df.stack().value_counts().sum()

skill_df.stack().value_counts().loc[skill_id] / response_count * 100

In [1]:
import keras
from keras.layers import Input, Dense, LSTM, TimeDistributed, Lambda, multiply
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

def build_skill2skill_model(input_shape, lstm_dim=32, dropout=0.0):
    input = Input(shape=input_shape, name='input skills')
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout,
                name='lstm_layer')(input)
    output = TimeDistributed(Dense(input_shape[-1], activation='softmax'), name='probability')(lstm)
    model = Model(inputs=[input], outputs=[output])
    adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def reduce_dim(x):
    x = K.max(x, axis=-1, keepdims=True)
    return x

def build_dkt_model(input_shape, lstm_dim=32, dropout=0.0):
    input_skills = Input(shape=input_shape, name='input skills')
    lstm = LSTM(lstm_dim, 
                return_sequences=True, 
                dropout=dropout,
                name='lstm_layer')(input_skills)
    dense = TimeDistributed(Dense(int(input_shape[-1]/2), activation='sigmoid'), name='probability_for_each')(lstm)
    
    skill_next = Input(shape=(input_shape[0], int(input_shape[1]/2)), name='next_skill_tested')
    merged = multiply([dense, skill_next], name='multiply')
    reduced = Lambda(reduce_dim, output_shape=(input_shape[0], 1), name='reduce_dim')(merged)
    
    model = Model(inputs=[input_skills, skill_next], outputs=[reduced])
    adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model



In [14]:

# Sample Usage: skill2skill
print('skill2skill')
skill2skill_model = build_skill2skill_model((99, skill_num), lstm_dim=64)

# Sample Usage: DKT
print('dkt')
dkt_model = build_dkt_model((99, 2 * skill_num), lstm_dim=64)
    

# train skill2skill
skill2skill_model.fit(skill_array[:, 0:-1], 
                      skill_array[:, 1:],
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

# train DKT 
dkt_model.fit([skill_response_array[:, 0:-1], skill_array[:, 1:]],
              response_array[:, 1:, np.newaxis],
              epochs=20, 
              batch_size=32, 
              shuffle=True,
              validation_split=0.2)

skill2skill
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 64)            45056     
                                                                 
 probability (TimeDistribute  (None, 99, 111)          7215      
 d)                                                              
                                                                 
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________
dkt
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input skills (InputLa

2022-03-15 13:40:17.768923: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-15 13:40:18.442970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:18.573053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:21.524521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - ETA: 0s - loss: 4.6260 - accuracy: 0.1267

2022-03-15 13:40:23.483354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:23.531510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 6s 102ms/step - loss: 4.6260 - accuracy: 0.1267 - val_loss: 4.5801 - val_accuracy: 0.1459
Epoch 2/20
15/15 [==============================] - 0s 29ms/step - loss: 4.1496 - accuracy: 0.1556 - val_loss: 4.4288 - val_accuracy: 0.0608
Epoch 3/20
15/15 [==============================] - 0s 30ms/step - loss: 3.6411 - accuracy: 0.1771 - val_loss: 4.2233 - val_accuracy: 0.1039
Epoch 4/20
15/15 [==============================] - 0s 30ms/step - loss: 3.2776 - accuracy: 0.2511 - val_loss: 3.9411 - val_accuracy: 0.1122
Epoch 5/20
15/15 [==============================] - 0s 28ms/step - loss: 2.9477 - accuracy: 0.3180 - val_loss: 3.5963 - val_accuracy: 0.1841
Epoch 6/20
15/15 [==============================] - 0s 29ms/step - loss: 2.6465 - accuracy: 0.3675 - val_loss: 3.3204 - val_accuracy: 0.2440
Epoch 7/20
15/15 [==============================] - 0s 29ms/step - loss: 2.3865 - accuracy: 0.4200 - val_loss: 3.0749 - val_accuracy: 0.3061
Epoch 8/20
15/15 [=====

2022-03-15 13:40:32.691607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:32.829020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:33.380333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - ETA: 0s - loss: 0.6711 - accuracy: 0.7261

2022-03-15 13:40:34.512911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 13:40:34.568647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 3s 81ms/step - loss: 0.6711 - accuracy: 0.7261 - val_loss: 0.6511 - val_accuracy: 0.8162
Epoch 2/20
15/15 [==============================] - 0s 29ms/step - loss: 0.5993 - accuracy: 0.8397 - val_loss: 0.5616 - val_accuracy: 0.8022
Epoch 3/20
15/15 [==============================] - 0s 29ms/step - loss: 0.4838 - accuracy: 0.8354 - val_loss: 0.4755 - val_accuracy: 0.8241
Epoch 4/20
15/15 [==============================] - 0s 29ms/step - loss: 0.4206 - accuracy: 0.8480 - val_loss: 0.4550 - val_accuracy: 0.8329
Epoch 5/20
15/15 [==============================] - 0s 29ms/step - loss: 0.3958 - accuracy: 0.8504 - val_loss: 0.4362 - val_accuracy: 0.8337
Epoch 6/20
15/15 [==============================] - 0s 29ms/step - loss: 0.3839 - accuracy: 0.8544 - val_loss: 0.4435 - val_accuracy: 0.8329
Epoch 7/20
15/15 [==============================] - 0s 29ms/step - loss: 0.3780 - accuracy: 0.8544 - val_loss: 0.4358 - val_accuracy: 0.8328
Epoch 8/20
15/15 [======

In [82]:
#Q2
skill2skill_model = build_skill2skill_model((99, skill_num), lstm_dim=64)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 64)            45056     
                                                                 
 probability (TimeDistribute  (None, 99, 111)          7215      
 d)                                                              
                                                                 
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________


In [85]:
X_train, X_test, y_train, y_test = train_test_split(skill_array[:, 0:-1], skill_array[:, 1:], test_size=0.30)

In [90]:
skill2skill_model.fit(X_train, 
                      y_train,
                      epochs=20, 
                      batch_size=32, 
                      shuffle=True,
                      validation_split=0.2)

Epoch 1/20


2022-03-15 16:07:39.648715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 16:07:39.772879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 16:07:39.930049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 4.6724 - accuracy: 0.0762

2022-03-15 16:07:41.297659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 16:07:41.347948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 3s 143ms/step - loss: 4.6724 - accuracy: 0.0762 - val_loss: 4.5937 - val_accuracy: 0.2305
Epoch 2/20
11/11 [==============================] - 0s 31ms/step - loss: 4.5161 - accuracy: 0.3038 - val_loss: 4.3455 - val_accuracy: 0.2533
Epoch 3/20
11/11 [==============================] - 0s 31ms/step - loss: 4.1546 - accuracy: 0.1553 - val_loss: 3.9098 - val_accuracy: 0.1296
Epoch 4/20
11/11 [==============================] - 0s 30ms/step - loss: 3.7465 - accuracy: 0.1944 - val_loss: 3.5026 - val_accuracy: 0.2302
Epoch 5/20
11/11 [==============================] - 0s 30ms/step - loss: 3.3494 - accuracy: 0.2455 - val_loss: 3.1314 - val_accuracy: 0.3234
Epoch 6/20
11/11 [==============================] - 0s 30ms/step - loss: 2.9759 - accuracy: 0.3461 - val_loss: 2.8104 - val_accuracy: 0.3755
Epoch 7/20
11/11 [==============================] - 0s 30ms/step - loss: 2.6561 - accuracy: 0.4195 - val_loss: 2.5514 - val_accuracy: 0.4209
Epoch 8/20
11/11 [=====

In [184]:
X_pred = skill2skill_model.predict(X_test)

In [253]:
X_pred_binary = X_pred[:]

for si, student in enumerate(X_pred_binary):
    for ri, response in enumerate(student):
        label = max(response)
        X_pred_binary[si][ri] = [1 if r == label else 0 for r in response]

In [254]:
from sklearn.metrics import accuracy_score

accuracy_score([b for a in y_test for b in a], [b for a in X_pred_binary for b in a])

0.7086776859504132

In [255]:
skills_correct = []
for i in range(len(X_pred_binary)):
    skills_correct.append([])
for si, student in enumerate(X_pred_binary):
    for ri, response in enumerate(student):
        skill_index = np.argmax(response)
        skills_correct[skill_index].append(1 if response[skill_index] == y_test[si][ri][skill_index] else 0)

In [299]:
skills_correct_means = [np.mean(i) for i in skills_correct]


/Users/elipleaner/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/elipleaner/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [316]:
df_skills_correct_means = pd.DataFrame(skills_correct_means, columns=['mean'])
df_skills_correct_means.sort_values(by='mean', ascending=False, inplace= True)
df_skills_correct_means.dropna(inplace=True)
df_skills_correct_means.reset_index(inplace=True)
df_skills_correct_means.rename(columns={'index': 'skill_id'}, inplace=True)
df_skills_correct_means

df_skills_correct_means['skill_name'] = df_skills_correct_means['skill_id'].map(lambda x: list(skill_dict.keys())[list(skill_dict.values()).index(x)])

In [318]:
# Q2 – What were the top 5 hardest and easiest to predict skills?
print(df_skills_correct_means.head())
print(df_skills_correct_means.tail())

   skill_id      mean                                     skill_name
0        50  1.000000                                      Unit Rate
1         5  0.909396                                   Scatter Plot
2        48  0.878378                                     Proportion
3        67  0.857143  Multiplication and Division Positive Decimals
4         7  0.851397                                          Table
    skill_id      mean                            skill_name
40        23  0.500000       Unit Conversion Within a System
41        39  0.484848                            Estimation
42        70  0.450777  Multiplication and Division Integers
43        86  0.402439               Order of Operations All
44        52  0.000000                    Divisibility Rules


## A note on Q2

Question 2 of the RNN homework ask you to report on the hardest and easiest to predict skills using the skill2skill model. In order to establish some sortable measure of prediction difficulty, you need to come up with a metric that represents this prediction difficulty. Here are a few suggestions for metrics

1) Accuracy: For each skill, what percent of the time (when it was the label) was it predicted correctly? One approach to this is to create a binary array for each skill, whose length is equal to the number of times the skill appears as a label. When that skill was the label and had the highest probability in the softmax output, the array value is 1, otherwise 0. The average value of this array is the accuracy of prediction for that skill. The finaly step is then to sort all skills by this average accuracy. 

2) Recall @ N: This is the same idea as above, except that you would look to see if the probability associated with the label's skill was among the top N probabilities. Say, for N = 5, if the label's probability was in the top N, the array value would be true, and false otherwise. The average of the array would then be the Recall @ N for that skill. This metric should always return a value equal to or higher  than accuracy.

3) Average probability: Instead of recording if the probability was the best, you could record only the probability itself. The skills will the lowest average probability is one way of characterizing them as being hardest (or least likely) to predict. 

In [328]:
X_train, X_test, y_train, y_test = train_test_split(skill_array[:, 0:-1], skill_array[:, 1:], test_size=0.30)

In [327]:
# lstm_dim: 128, batch_size: 8, validation_split: 0
# accuracy: 0.8475091827364555
for lstm_dim in [64, 128]:
    for batch_size in [8, 16]:
        for validation_split in [0, 0.1, 0.2, 0.3]:
            skill2skill_model = build_skill2skill_model((99, skill_num), lstm_dim=lstm_dim)

            skill2skill_model.fit(X_train, 
                                  y_train,
                                  epochs=20, 
                                  batch_size=batch_size, 
                                  shuffle=True,
                                  validation_split=validation_split, verbose=0)

            y_pred = skill2skill_model.predict(X_test)

            y_pred_binary = y_pred[:]

            for si, student in enumerate(y_pred_binary):
                for ri, response in enumerate(student):
                    label = max(response)
                    y_pred_binary[si][ri] = [1 if r == label else 0 for r in response]

            print('lstm_dim: %s, batch_size: %s, validation_split: %s' % (lstm_dim, batch_size, validation_split))
            print('accuracy: %s' % accuracy_score([b for a in y_test for b in a], [b for a in y_pred_binary for b in a]))

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 64)            45056     
                                                                 
 probability (TimeDistribute  (None, 99, 111)          7215      
 d)                                                              
                                                                 
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________


2022-03-16 15:09:14.205797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:14.335932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:14.574147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:38.429692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:38.473876: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


lstm_dim: 64, batch_size: 8, validation_split: 0
accuracy: 0.8370064279155188
Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 64)            45056     
                                                                 
 probability (TimeDistribute  (None, 99, 111)          7215      
 d)                                                              
                                                                 
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________


2022-03-16 15:09:39.984215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:40.120546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:40.337613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:42.514402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:09:42.573268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:10:07.533751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:10:07.646847: I tensorflow/core/grappler/optimizers/cust

lstm_dim: 64, batch_size: 8, validation_split: 0.1
accuracy: 0.8335629017447199
Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 64)            45056     
                                                                 
 probability (TimeDistribute  (None, 99, 111)          7215      
 d)                                                              
                                                                 
Total params: 52,271
Trainable params: 52,271
Non-trainable params: 0
_________________________________________________________________


...

lstm_dim: 128, batch_size: 16, validation_split: 0.1
accuracy: 0.8251262626262627
Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 128)           122880    
                                                                 
 probability (TimeDistribute  (None, 99, 111)          14319     
 d)                                                              
                                                                 
Total params: 137,199
Trainable params: 137,199
Non-trainable params: 0
_________________________________________________________________


2022-03-16 15:14:26.117503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:26.259244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:26.496170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:27.958969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:28.013131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:40.201924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:40.242467: I tensorflow/core/grappler/optimizers/cust

lstm_dim: 128, batch_size: 16, validation_split: 0.2
accuracy: 0.8097451790633609
Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input skills (InputLayer)   [(None, 99, 111)]         0         
                                                                 
 lstm_layer (LSTM)           (None, 99, 128)           122880    
                                                                 
 probability (TimeDistribute  (None, 99, 111)          14319     
 d)                                                              
                                                                 
Total params: 137,199
Trainable params: 137,199
Non-trainable params: 0
_________________________________________________________________


2022-03-16 15:14:41.677219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:41.836423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:42.076767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:43.641526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:43.694018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:55.297974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-16 15:14:55.343752: I tensorflow/core/grappler/optimizers/cust

lstm_dim: 128, batch_size: 16, validation_split: 0.3
accuracy: 0.7976354453627181


In [373]:
# x: [skill_response_array[:, 0:-1], skill_array[:, 1:]],
# y: response_array[:, 1:, np.newaxis]

In [8]:
test_size = 0.3

idx_split = np.array(range(len(skill_response_array[:, 0:-1])))
np.random.shuffle(idx_split)

X_train = skill_response_array[:, 0:-1][idx_split[int(test_size * len(idx_split)):]]
y_train = response_array[:, 1:, np.newaxis][idx_split[int(test_size * len(idx_split)):]]
skills_train = skill_array[:, 1:][idx_split[int(test_size * len(idx_split)):]]

X_test = skill_response_array[:, 0:-1][idx_split[:int(test_size * len(idx_split))]]
y_test = response_array[:, 1:, np.newaxis][idx_split[:int(test_size * len(idx_split))]]
skills_test = skill_array[:, 1:][idx_split[:int(test_size * len(idx_split))]]

In [11]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [12]:
dkt_model = build_dkt_model((99, 2 * skill_num))

# train DKT 
dkt_model.fit([X_train, skills_train],
              response_array[:, 1:, np.newaxis],
              epochs=20, 
              batch_size=32, 
              shuffle=True,
              validation_split=0.2)
            
y_pred = dkt_model.predict([X_test, skills_test])

y_pred_binary = y_pred[:]

pred_auc = [b[0] for a in y_pred for b in a]

for si, student in enumerate(y_pred_binary):
    for ri, response in enumerate(student):
        y_pred_binary[si][ri][0] = round(y_pred_binary[si][ri][0])

y = [b[0] for a in y_test for b in a]
pred = [b[0] for a in y_pred_binary for b in a]

print('accuracy: %s' % (accuracy_score(y, pred)))
print('auc: %s' % (roc_auc_score(y, pred)))

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input skills (InputLayer)      [(None, 99, 222)]    0           []                               
                                                                                                  
 lstm_layer (LSTM)              (None, 99, 32)       32640       ['input skills[0][0]']           
                                                                                                  
 probability_for_each (TimeDist  (None, 99, 111)     3663        ['lstm_layer[0][0]']             
 ributed)                                                                                         
                                                                                                  
 next_skill_tested (InputLayer)  [(None, 99, 111)]   0           []                         

2022-03-28 20:55:00.576764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-28 20:55:00.710050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/11 [=>............................] - ETA: 11s - loss: 0.6912 - accuracy: 0.5581

2022-03-28 20:55:00.842023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 0.6825 - accuracy: 0.6966

2022-03-28 20:55:01.656953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-28 20:55:01.712560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 2s 90ms/step - loss: 0.6825 - accuracy: 0.6966 - val_loss: 0.6714 - val_accuracy: 0.7959
Epoch 2/20
11/11 [==============================] - 0s 27ms/step - loss: 0.6608 - accuracy: 0.8190 - val_loss: 0.6480 - val_accuracy: 0.8235
Epoch 3/20
11/11 [==============================] - 0s 27ms/step - loss: 0.6270 - accuracy: 0.8338 - val_loss: 0.6062 - val_accuracy: 0.8001
Epoch 4/20
11/11 [==============================] - 0s 26ms/step - loss: 0.5671 - accuracy: 0.8124 - val_loss: 0.5458 - val_accuracy: 0.8142
Epoch 5/20
11/11 [==============================] - 0s 25ms/step - loss: 0.5027 - accuracy: 0.8415 - val_loss: 0.4995 - val_accuracy: 0.8386
Epoch 6/20
11/11 [==============================] - 0s 26ms/step - loss: 0.4633 - accuracy: 0.8489 - val_loss: 0.4763 - val_accuracy: 0.8402
Epoch 7/20
11/11 [==============================] - 0s 25ms/step - loss: 0.4456 - accuracy: 0.8500 - val_loss: 0.4634 - val_accuracy: 0.8402
Epoch 8/20
11/11 [======

2022-03-28 20:55:07.422326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-28 20:55:07.458769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


accuracy: 0.8510822510822511
auc: 0.5010858551582025


In [460]:
models = []

for lstm_dim in [64, 128]:
    for batch_size in [8, 16, 32]:
        for validation_split in [0, 0.1, 0.2, 0.3]:
            dkt_model = build_dkt_model((99, 2 * skill_num), lstm_dim=lstm_dim)

            dkt_model.fit([X_train, skills_train],
              y_train,
              epochs=20, 
              batch_size=batch_size, 
              shuffle=True,
              validation_split=validation_split)
            
            y_pred = dkt_model.predict([X_test, skills_test])
            
            y_pred_binary = y_pred[:]

            pred_auc = [b[0] for a in y_pred for b in a]

            for si, student in enumerate(y_pred_binary):
                for ri, response in enumerate(student):
                    y_pred_binary[si][ri][0] = round(y_pred_binary[si][ri][0])

            y = [b[0] for a in y_test for b in a]
            pred = [b[0] for a in y_pred_binary for b in a]
            
            models.append({'lstm_dim': lstm_dim, 'batch_size': batch_size, 'validation_split': validation_split, 'accuracy': accuracy_score(y, pred), 'auc': roc_auc_score(y, pred)}) 
            
            print('lstm_dim: %s | batch_size: %s | validation_split: %s' % (lstm_dim, batch_size, validation_split))
            print('accuracy: %s' % (accuracy_score(y, pred)))
            print('auc: %s' % (roc_auc_score(y, pred)))
            

Model: "model_45"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input skills (InputLayer)      [(None, 99, 222)]    0           []                               
                                                                                                  
 lstm_layer (LSTM)              (None, 99, 64)       73472       ['input skills[0][0]']           
                                                                                                  
 probability_for_each (TimeDist  (None, 99, 111)     7215        ['lstm_layer[0][0]']             
 ributed)                                                                                         
                                                                                                  
 next_skill_tested (InputLayer)  [(None, 99, 111)]   0           []                        

2022-03-18 11:46:36.186365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-18 11:46:36.356237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-18 11:46:36.615940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


52/52 [==============================] - 3s 36ms/step - loss: 0.5756 - accuracy: 0.7881
Epoch 2/20
52/52 [==============================] - 1s 27ms/step - loss: 0.4103 - accuracy: 0.8474
Epoch 3/20
52/52 [==============================] - 1s 28ms/step - loss: 0.3872 - accuracy: 0.8516
Epoch 4/20
52/52 [==============================] - 1s 28ms/step - loss: 0.3779 - accuracy: 0.8548
Epoch 5/20
52/52 [==============================] - 1s 28ms/step - loss: 0.3706 - accuracy: 0.8551
Epoch 6/20
52/52 [==============================] - 1s 28ms/step - loss: 0.3640 - accuracy: 0.8569
Epoch 7/20
52/52 [==============================] - 1s 29ms/step - loss: 0.4005 - accuracy: 0.8389
Epoch 8/20
52/52 [==============================] - 1s 26ms/step - loss: 0.3714 - accuracy: 0.8559
Epoch 9/20
52/52 [==============================] - 1s 26ms/step - loss: 0.3653 - accuracy: 0.8558
Epoch 10/20
52/52 [==============================] - 1s 26ms/step - loss: 0.3581 - accuracy: 0.8580
Epoch 11/20
52/52 [=

2022-03-18 11:47:04.880698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-18 11:47:04.928637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


lstm_dim: 64 | batch_size: 8 | validation_split: 0
accuracy: 0.8555844155844156
auc: 0.6020983833942904
Model: "model_46"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input skills (InputLayer)      [(None, 99, 222)]    0           []                               
                                                                                                  
 lstm_layer (LSTM)              (None, 99, 64)       73472       ['input skills[0][0]']           
                                                                                                  
 probability_for_each (TimeDist  (None, 99, 111)     7215        ['lstm_layer[0][0]']             
 ributed)                                                                                         
                                                                                      

...

9/9 [==============================] - ETA: 0s - loss: 0.6660 - accuracy: 0.7432

2022-03-18 11:53:59.017417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-18 11:53:59.094373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 4s 222ms/step - loss: 0.6660 - accuracy: 0.7432 - val_loss: 0.6328 - val_accuracy: 0.8362
Epoch 2/20
9/9 [==============================] - 0s 52ms/step - loss: 0.5889 - accuracy: 0.8312 - val_loss: 0.5235 - val_accuracy: 0.8261
Epoch 3/20
9/9 [==============================] - 0s 44ms/step - loss: 0.4712 - accuracy: 0.8368 - val_loss: 0.4327 - val_accuracy: 0.8480
Epoch 4/20
9/9 [==============================] - 0s 39ms/step - loss: 0.4157 - accuracy: 0.8489 - val_loss: 0.4031 - val_accuracy: 0.8532
Epoch 5/20
9/9 [==============================] - 0s 42ms/step - loss: 0.3985 - accuracy: 0.8534 - val_loss: 0.3944 - val_accuracy: 0.8540
Epoch 6/20
9/9 [==============================] - 0s 36ms/step - loss: 0.3909 - accuracy: 0.8537 - val_loss: 0.3910 - val_accuracy: 0.8540
Epoch 7/20
9/9 [==============================] - 0s 36ms/step - loss: 0.3846 - accuracy: 0.8549 - val_loss: 0.3856 - val_accuracy: 0.8554
Epoch 8/20
9/9 [=====================

2022-03-18 11:54:06.303115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-18 11:54:06.345301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


lstm_dim: 128 | batch_size: 32 | validation_split: 0.3
accuracy: 0.845945165945166
auc: 0.5524961488737092


In [461]:
models

 # {'lstm_dim': 128,
 #  'batch_size': 8,
 #  'validation_split': 0.3,
 #  'accuracy': 0.8536219336219336,
 #  'auc': 0.6187421536171677}

[{'lstm_dim': 64,
  'batch_size': 8,
  'validation_split': 0,
  'accuracy': 0.8555844155844156,
  'auc': 0.6020983833942904},
 {'lstm_dim': 64,
  'batch_size': 8,
  'validation_split': 0.1,
  'accuracy': 0.855007215007215,
  'auc': 0.5908646420518842},
 {'lstm_dim': 64,
  'batch_size': 8,
  'validation_split': 0.2,
  'accuracy': 0.8531024531024531,
  'auc': 0.5912808766112402},
 {'lstm_dim': 64,
  'batch_size': 8,
  'validation_split': 0.3,
  'accuracy': 0.8535642135642135,
  'auc': 0.5881632414969955},
 {'lstm_dim': 64,
  'batch_size': 16,
  'validation_split': 0,
  'accuracy': 0.8531601731601731,
  'auc': 0.5866488959877879},
 {'lstm_dim': 64,
  'batch_size': 16,
  'validation_split': 0.1,
  'accuracy': 0.8525252525252526,
  'auc': 0.5882488693215657},
 {'lstm_dim': 64,
  'batch_size': 16,
  'validation_split': 0.2,
  'accuracy': 0.8526406926406926,
  'auc': 0.5778536344122156},
 {'lstm_dim': 64,
  'batch_size': 16,
  'validation_split': 0.3,
  'accuracy': 0.8506204906204906,
  'auc'

accuracy: 0.8462914862914863
auc: 0.5469055107584941
